# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import httpx
from selectolax.parser import HTMLParser
import re

# Get Data From vlr.gg

In [63]:
base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'

res = httpx.get(base_url+event_url)

html = HTMLParser(res.text)

matches = html.css('a.wf-module-item')

completed_matches = [match for match in matches if match.css_first('div.ml-status').text() == 'Completed']

for match in completed_matches[:2]:
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '

    print(match_sum:=match_result.join(match_name))

    tables = match_html.css('table.wf-table-inset')

    for table in tables:
        for t in pd.read_html(table.html):
            sample_df = t
            for key,df in get_clean_df(t).items():
                pass

KOI 0:2 NRG Esports
DetonatioN FocusMe 0:2 Giants Gaming


# Cleaning Data

In [53]:
def get_default(serie,func,dtype,np.nan):
    
    try:
        return serie.map(func).map(dtype)
    except AttributeError as e:
        return pd.Series([default] * serie.shape[0])

In [54]:
series = pd.Series([1,2,3,4,5])
get_default(series,0,int)

'int' object has no attribute 'split'


0    0
1    0
2    0
3    0
4    0
dtype: int64

In [66]:
def temp_clean_df(df,func):
    result_df = pd.DataFrame()
    
    result_df['Name'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[0])
    result_df['Team'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[1])
    result_df['R'] = get_default(df['R'],func,float)
    result_df['ACS'] = get_default(df['ACS'],func,int)
    result_df['K'] = get_default(df['K'],func,int)
    result_df['D'] = df['D'].map(lambda d : d[1:-1].strip()).map(func).map(int)
    result_df['A'] = df['A'].map(func).map(int)
    result_df['KD_Diff'] = df['+/–'].map(func).map(lambda d : int(d))
    result_df['KAST%'] = df['KAST'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['ADR'] = df['ADR'].map(func).map(int)
    result_df['HS%'] = df['HS%'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['FK'] = df['FK'].map(func).map(int)
    result_df['FD'] = df['FD'].map(func).map(int)
    result_df['FKFD_Diff'] = df['+/–.1'].map(func).map(int)
    
    
    
    return result_df

In [12]:
def clean_df(df,func):
    result_df = pd.DataFrame()
    
    result_df['Name'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[0])
    result_df['Team'] = df['Unnamed: 0'].map(lambda n : n.split(' ')[1])
    result_df['R'] = df['R'].map(func).map(float)
    result_df['ACS'] = df['ACS'].map(func).map(int)
    result_df['K'] = df['K'].map(func).map(int)
    result_df['D'] = df['D'].map(lambda d : d[1:-1].strip()).map(func).map(int)
    result_df['A'] = df['A'].map(func).map(int)
    result_df['KD_Diff'] = df['+/–'].map(func).map(lambda d : int(d))
    result_df['KAST%'] = df['KAST'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['ADR'] = df['ADR'].map(func).map(int)
    result_df['HS%'] = df['HS%'].map(func).map(lambda p : int(p[:-1])/100)
    result_df['FK'] = df['FK'].map(func).map(int)
    result_df['FD'] = df['FD'].map(func).map(int)
    result_df['FKFD_Diff'] = df['+/–.1'].map(func).map(int)
    
    
    
    return result_df

In [13]:
def get_clean_df(df):
    result = {}
    def get_all(s):
        try:
            return s.split(' ')[0]
        except AttributeError:
            return '00'
    
    def get_atk(s):
        try:
            return s.split(' ')[2]
        except AttributeError:
            return '00'
    
    def get_def(s):
        try:
            return s.split(' ')[4]
        except AttributeError:
            return '00'

    for side in ['all','atk','def']:
        result[side] = clean_df(df,eval(f'get_{side}'))
        
    return result

# Testing

In [112]:
sample_match = completed_matches[20]

In [166]:
match_res = httpx.get(match_url := f'{base_url}{sample_match.attributes["href"]}')

match_html = HTMLParser(match_res.text)

match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

messy_match_result = match_html.css_first('div.js-spoiler').text()
match_result = ' ' + ''.join(re.findall("\S+",messy_match_result)) + ' '
print(match_sum:=match_result.join(match_name))

tables = match_html.css('table.wf-table-inset')

for table in tables:
    for t in pd.read_html(table.html):
        sample_df = t
        for key,df in get_clean_df(t).items():
            display(df.head(3))

Sentinels 0:2 FNATIC


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,TenZ,SEN,1.11,193,14,13,3,1,0.63,125,0.20,2,5,-3
1,Sacy,SEN,0.72,182,13,16,7,-3,0.74,115,0.32,1,2,-1
2,pANcada,SEN,0.72,157,11,17,6,-6,0.58,112,0.33,0,1,-1


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,TenZ,SEN,1.29,247,11,8,2,3,0.67,164,0.19,2,4,-2
1,Sacy,SEN,0.68,152,7,12,5,-5,0.67,85,0.35,0,1,-1
2,pANcada,SEN,0.70,147,6,11,5,-5,0.58,105,0.26,0,1,-1


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,TenZ,SEN,0.80,102,3,5,1,-2,0.57,59,0.25,0,1,-1
1,Sacy,SEN,0.80,235,6,4,2,2,0.86,167,0.30,1,1,0
2,pANcada,SEN,0.75,176,5,6,1,-1,0.57,125,0.43,0,0,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Leo,FNC,1.35,274,17,12,8,5,0.89,175,0.20,2,0,2
1,Chronicle,FNC,1.33,252,17,12,15,5,0.89,125,0.23,5,0,5
2,Derke,FNC,1.33,286,20,11,2,9,0.68,192,0.33,3,3,0


...

,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,pANcada,SEN,0.74,223,9,9,2,0,0.75,149,0.50,0,1,-1
1,TenZ,SEN,1.08,188,8,7,2,1,0.50,139,0.41,2,1,1
2,Sacy,SEN,0.83,150,7,8,1,-1,0.75,88,0.70,1,1,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,pANcada,SEN,1.33,268,8,4,0,4,0.88,169,0.41,1,0,1
1,TenZ,SEN,0.53,155,4,7,3,-3,0.75,90,0.06,1,0,1
2,Sacy,SEN,0.66,179,4,7,2,-3,0.63,146,0.25,1,3,-2


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Derke,FNC,1.41,317,23,12,0,11,0.80,188,0.27,6,3,3
1,Alfajer,FNC,1.31,231,17,11,3,6,0.85,152,0.32,1,1,0
2,Leo,FNC,1.13,123,9,7,4,2,0.75,71,0.16,0,0,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Derke,FNC,1.32,344,10,5,0,5,0.75,198,0.19,1,2,-1
1,Alfajer,FNC,1.10,223,7,6,2,1,0.88,143,0.26,1,0,1
2,Leo,FNC,1.45,157,4,2,4,2,0.88,89,0.44,0,0,0


,Name,Team,R,ACS,K,D,A,KD_Diff,KAST%,ADR,HS%,FK,FD,FKFD_Diff
0,Derke,FNC,1.47,301,13,7,0,6,0.83,181,0.39,5,1,4
1,Alfajer,FNC,1.45,237,10,5,1,5,0.83,158,0.37,0,1,-1
2,Leo,FNC,0.92,101,5,5,0,0,0.67,59,0.08,0,0,0


In [106]:
sample_df = sample_df.fillna('0').astype('object')

In [114]:
sample_df

,Unnamed: 0,Unnamed: 1,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Derke FNC,NaN,1.41 1.32 1.47,317 344 301,23 10 13,/ 12 5 7 /,0 0 0,+11 +5 +6,80% 75% 83%,188 198 181,27% 19% 39%,6 1 5,3 2 1,+3 -1 +4
1,Alfajer FNC,NaN,1.31 1.10 1.45,231 223 237,17 7 10,/ 11 6 5 /,3 2 1,+6 +1 +5,85% 88% 83%,152 143 158,32% 26% 37%,1 1 0,1 0 1,0 +1 -1
2,Leo FNC,NaN,1.13 1.45 0.92,123 157 101,9 4 5,/ 7 2 5 /,4 4 0,+2 +2 0,75% 88% 67%,71 89 59,16% 44% 8%,0 0 0,0 0 0,0 0 0
3,Chronicle FNC,NaN,1.01 1.51 0.68,195 203 190,14 6 8,/ 14 3 11 /,1 1 0,0 +3 -3,70% 88% 58%,120 111 126,13% 11% 15%,1 0 1,2 0 2,-1 0 -1
4,Boaster FNC,NaN,0.97 0.83 1.07,130 115 140,8 3 5,/ 12 5 7 /,7 2 5,-4 -2 -2,80% 88% 75%,83 52 103,25% 40% 21%,3 2 1,3 2 1,0 0 0


In [102]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5 non-null      object
 1   Unnamed: 1  5 non-null      object
 2   R           5 non-null      object
 3   ACS         5 non-null      object
 4   K           5 non-null      object
 5   D           5 non-null      object
 6   A           5 non-null      object
 7   +/–         5 non-null      object
 8   KAST        5 non-null      object
 9   ADR         5 non-null      object
 10  HS%         5 non-null      object
 11  FK          5 non-null      object
 12  FD          5 non-null      object
 13  +/–.1       5 non-null      object
dtypes: object(14)
memory usage: 692.0+ bytes


In [110]:
sample_df['D'].map(lambda x : re.findall("[\d+-]+",x,re.A))

0    [14]
1    [15]
2    [16]
3    [17]
4    [13]
Name: D, dtype: object

In [105]:
sample_df['R']

0    0
1    0
2    0
3    0
4    0
Name: R, dtype: object

## Sub Column Approach

In [167]:
sample_df

,Unnamed: 0,Unnamed: 1,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Derke FNC,NaN,1.41 1.32 1.47,317 344 301,23 10 13,/ 12 5 7 /,0 0 0,+11 +5 +6,80% 75% 83%,188 198 181,27% 19% 39%,6 1 5,3 2 1,+3 -1 +4
1,Alfajer FNC,NaN,1.31 1.10 1.45,231 223 237,17 7 10,/ 11 6 5 /,3 2 1,+6 +1 +5,85% 88% 83%,152 143 158,32% 26% 37%,1 1 0,1 0 1,0 +1 -1
2,Leo FNC,NaN,1.13 1.45 0.92,123 157 101,9 4 5,/ 7 2 5 /,4 4 0,+2 +2 0,75% 88% 67%,71 89 59,16% 44% 8%,0 0 0,0 0 0,0 0 0
3,Chronicle FNC,NaN,1.01 1.51 0.68,195 203 190,14 6 8,/ 14 3 11 /,1 1 0,0 +3 -3,70% 88% 58%,120 111 126,13% 11% 15%,1 0 1,2 0 2,-1 0 -1
4,Boaster FNC,NaN,0.97 0.83 1.07,130 115 140,8 3 5,/ 12 5 7 /,7 2 5,-4 -2 -2,80% 88% 75%,83 52 103,25% 40% 21%,3 2 1,3 2 1,0 0 0


In [168]:
name = sample_df.iloc[:,0].map(lambda s : s.split(' ')[0])
team = sample_df.iloc[:,0].map(lambda s : s.split(' ')[1])

In [169]:
sample_df.iloc[:,0] = name
sample_df.iloc[:,1] = team
display(sample_df)

,Unnamed: 0,Unnamed: 1,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Derke,FNC,1.41 1.32 1.47,317 344 301,23 10 13,/ 12 5 7 /,0 0 0,+11 +5 +6,80% 75% 83%,188 198 181,27% 19% 39%,6 1 5,3 2 1,+3 -1 +4
1,Alfajer,FNC,1.31 1.10 1.45,231 223 237,17 7 10,/ 11 6 5 /,3 2 1,+6 +1 +5,85% 88% 83%,152 143 158,32% 26% 37%,1 1 0,1 0 1,0 +1 -1
2,Leo,FNC,1.13 1.45 0.92,123 157 101,9 4 5,/ 7 2 5 /,4 4 0,+2 +2 0,75% 88% 67%,71 89 59,16% 44% 8%,0 0 0,0 0 0,0 0 0
3,Chronicle,FNC,1.01 1.51 0.68,195 203 190,14 6 8,/ 14 3 11 /,1 1 0,0 +3 -3,70% 88% 58%,120 111 126,13% 11% 15%,1 0 1,2 0 2,-1 0 -1
4,Boaster,FNC,0.97 0.83 1.07,130 115 140,8 3 5,/ 12 5 7 /,7 2 5,-4 -2 -2,80% 88% 75%,83 52 103,25% 40% 21%,3 2 1,3 2 1,0 0 0


In [170]:
sample_df=sample_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})

In [171]:
sample_df.iloc[:,2:] = sample_df.iloc[:,2:].applymap(lambda x :  ' '.join(re.findall("[\d+-.]+",x,re.A)))
sample_df

,Name,Team,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Derke,FNC,1.41 1.32 1.47,317 344 301,23 10 13,12 5 7,0 0 0,+11 +5 +6,80 75 83,188 198 181,27 19 39,6 1 5,3 2 1,+3 -1 +4
1,Alfajer,FNC,1.31 1.10 1.45,231 223 237,17 7 10,11 6 5,3 2 1,+6 +1 +5,85 88 83,152 143 158,32 26 37,1 1 0,1 0 1,0 +1 -1
2,Leo,FNC,1.13 1.45 0.92,123 157 101,9 4 5,7 2 5,4 4 0,+2 +2 0,75 88 67,71 89 59,16 44 8,0 0 0,0 0 0,0 0 0
3,Chronicle,FNC,1.01 1.51 0.68,195 203 190,14 6 8,14 3 11,1 1 0,0 +3 -3,70 88 58,120 111 126,13 11 15,1 0 1,2 0 2,-1 0 -1
4,Boaster,FNC,0.97 0.83 1.07,130 115 140,8 3 5,12 5 7,7 2 5,-4 -2 -2,80 88 75,83 52 103,25 40 21,3 2 1,3 2 1,0 0 0


In [203]:
new_columns = []
for c in sample_df.columns[2:]:
    for side in ['all','atk','def']:
        new_columns.append((c,side))
    
new_columns = np.array(new_columns).reshape(-1,3,2)

In [204]:
for i in new_columns:
    print(i)

[['R' 'all']
 ['R' 'atk']
 ['R' 'def']]
[['ACS' 'all']
 ['ACS' 'atk']
 ['ACS' 'def']]
[['K' 'all']
 ['K' 'atk']
 ['K' 'def']]
[['D' 'all']
 ['D' 'atk']
 ['D' 'def']]
[['A' 'all']
 ['A' 'atk']
 ['A' 'def']]
[['+/–' 'all']
 ['+/–' 'atk']
 ['+/–' 'def']]
[['KAST' 'all']
 ['KAST' 'atk']
 ['KAST' 'def']]
[['ADR' 'all']
 ['ADR' 'atk']
 ['ADR' 'def']]
[['HS%' 'all']
 ['HS%' 'atk']
 ['HS%' 'def']]
[['FK' 'all']
 ['FK' 'atk']
 ['FK' 'def']]
[['FD' 'all']
 ['FD' 'atk']
 ['FD' 'def']]
[['+/–.1' 'all']
 ['+/–.1' 'atk']
 ['+/–.1' 'def']]


In [186]:
sample_df

,Name,Team,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1
0,Derke,FNC,1.41 1.32 1.47,317 344 301,23 10 13,12 5 7,0 0 0,+11 +5 +6,80 75 83,188 198 181,27 19 39,6 1 5,3 2 1,+3 -1 +4
1,Alfajer,FNC,1.31 1.10 1.45,231 223 237,17 7 10,11 6 5,3 2 1,+6 +1 +5,85 88 83,152 143 158,32 26 37,1 1 0,1 0 1,0 +1 -1
2,Leo,FNC,1.13 1.45 0.92,123 157 101,9 4 5,7 2 5,4 4 0,+2 +2 0,75 88 67,71 89 59,16 44 8,0 0 0,0 0 0,0 0 0
3,Chronicle,FNC,1.01 1.51 0.68,195 203 190,14 6 8,14 3 11,1 1 0,0 +3 -3,70 88 58,120 111 126,13 11 15,1 0 1,2 0 2,-1 0 -1
4,Boaster,FNC,0.97 0.83 1.07,130 115 140,8 3 5,12 5 7,7 2 5,-4 -2 -2,80 88 75,83 52 103,25 40 21,3 2 1,3 2 1,0 0 0


In [255]:
new_sample_df = pd.DataFrame(sample_df.iloc[:,:2])

In [256]:
new_sample_df.columns = [('Name','Name'),('Team','Team')]

In [257]:
for nc,c in zip(new_columns,sample_df.iloc[:,2:].columns):
    new_sample_df[list(map(lambda x: tuple(x),list(nc)))] = sample_df[c].str.split(' ',expand=True)

In [258]:
new_sample_df.columns = pd.MultiIndex.from_tuples(new_sample_df.columns,name=['Type','Side'])

In [259]:
new_sample_df

Type       Name Team     R              ACS             K      ... HS%  FK  \
Side       Name Team   all   atk   def  all  atk  def all atk  ... def all   
0         Derke  FNC  1.41  1.32  1.47  317  344  301  23  10  ...  39   6   
1       Alfajer  FNC  1.31  1.10  1.45  231  223  237  17   7  ...  37   1   
2           Leo  FNC  1.13  1.45  0.92  123  157  101   9   4  ...   8   0   
3     Chronicle  FNC  1.01  1.51  0.68  195  203  190  14   6  ...  15   1   
4       Boaster  FNC  0.97  0.83  1.07  130  115  140   8   3  ...  21   3   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   5   3   2   1    +3  -1  +4  
1      1   0   1   0   1     0  +1  -1  
2      0   0   0   0   0     0   0   0  
3      0   1   2   0   2    -1   0  -1  
4      2   1   3   2   1     0   0   0  

[5 rows x 38 columns]